In [7]:
%%sh

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at 2f40a483d7 [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [8]:
import torch
# From the torchvision references we cloned
import transforms as T
import fiftyone as fo
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import skimage.io as io
import fiftyone as fo
import pycocotools 
import glob
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm_notebook
from pycocotools import mask as maskUtils
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [9]:
CHECKPOINT=True
NUM_CLASSES=2

In [10]:
%sx?

Docstring:
Shell execute - run shell command and capture output (!! is short-hand).

%sx command

IPython will run the given command using commands.getoutput(), and
return the result formatted as a list (split on '\n').  Since the
output is _returned_, it will be stored in ipython's regular output
cache Out[N] and in the '_N' automatic variables.

Notes:

1) If an input line begins with '!!', then %sx is automatically
invoked.  That is, while::

  !ls

causes ipython to simply issue system('ls'), typing::

  !!ls

is a shorthand equivalent to::

  %sx ls

2) %sx differs from %sc in that %sx automatically splits into a list,
like '%sc -l'.  The reason for this is to make it as easy as possible
to process line-oriented shell output via further python commands.
%sc is meant to provide much finer control, but requires more
typing.

3) Just like %sc -l, this is a list with special attributes:
::

  .l (or .list) : value as list.
  .n (or .nlstr): value as newline-separated string.
  .s (or 

In [ ]:
import torchvision.transforms as transforms
# Import required libraries
import torch
import cv2
import os

In [ ]:
# import os
from pathlib import Path

dir_path = r'../validation-dataset/'
def scan_FF(scan_path):
    count = 0
    
    for path in os.scandir(scan_path):
        if path.is_file():
            # file_path = scan_path+"/"+path
            print("File:",os.path.abspath(path))
            count += 1
        elif os.path.isdir(path):
            scan_FF(path)
    # print('directory:',scan_path,'file count:', count) 

scan_FF(dir_path)

In [ ]:
import os
from glob import glob
import pandas as pd

PATH = r'../validation-dataset/'
EXT = "*.jpg"
all_jpg_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
# print(all_jpg_files[2])

In [ ]:
images = pd.DataFrame(columns = ['color','3cr','vehicle','filename','budget_id','file_path'])
images['file_path'] = all_jpg_files
images['budget_id'] = images.file_path.apply(lambda x: x.split("/")[3])
images['filename'] = images.file_path.apply(lambda x: x.split("/")[4])
images['color'] = images.file_path.apply(lambda x: x.split("/")[2])
images = images.drop(['vehicle'], axis='columns')
images.drop('file_path', axis='columns')

In [ ]:
csv_path = '../csvs/images.csv'
filepath = Path(csv_path)
images.to_csv(filepath)

In [ ]:
import torchvision.transforms as transforms
# Import required libraries
import torch
import cv2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
img_dir=root_path+"/dataset/inference/"
img_converted = cv2.imread(img_dir)
  
# Convert BGR image to RGB image
image = cv2.cvtColor(img_converted, cv2.COLOR_BGR2RGB)
transform = transforms.Compose([
    transforms.ToTensor()
])
  
# transform = transforms.PILToTensor()
# Convert the PIL image to Torch tensor
img = transform(image)
print(type(img))
model.eval()
with torch.no_grad():
  prediction=model([img.to(device)])

In [ ]:
prediction[0]['labels']
#type(img)

In [ ]:
QUANT_DETECTION=len(prediction[0]['masks'])

In [ ]:
QUANT_DETECTION

In [ ]:
imagem=Image.fromarray(img.mul(255).permute(1,2,0).byte().numpy())
mask_predicted=Image.fromarray(prediction[0]['masks'][2,0].mul(255).byte().cpu().numpy())
blank = imagem.point(lambda _: 0)

In [ ]:
pred = prediction[0]
len(pred['scores'])

In [ ]:
pred['scores']

In [ ]:
trMASK=0.40
confidence=0.8
x=0
img = imagem
width, height = img.size
pred = prediction[0]
colors=['red','blue','green','yellow','gray','orange','pink','brown','purple','white','black','beige','darkgray','darkred','darkblue','darkgreen','darkyellow']
print(len(colors))
if len(pred['labels']) > 0:
    # print (len(pred['labels']))
    mask_labels=[]
    count_cor=0
    for n in range(len(pred['labels'])):
      print(pred['scores'][n])
      if pred['scores'][n]>=confidence:
        if not pred['labels'][n] in mask_labels:
          mask_labels.append(pred['labels'][n])
          mask = pred['masks'].cpu().numpy()[n][0]>trMASK
          mask = (mask*200).astype(np.uint8)
          #print(len(mask_labels))
          
          mask = Image.fromarray(mask).convert('L')
          img2 = Image.new(mode='RGB',size=(width, height),color=colors[len(mask_labels)])
          count_cor+=1

          img = Image.composite(img2,img,mask)

In [ ]:
Image.fromarray(prediction[0]['masks'][0,0].mul(255).byte().cpu().numpy())

In [ ]:
session = fo.launch_app(data)

In [ ]:
import torch
import fiftyone.utils.coco as fouc
from PIL import Image


class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.
    
    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for training or testing
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset that contains the 
            desired labels to load
        classes (None): a list of class strings that are used to define the mapping between
            class names and indices. If None, it will use all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="segmentations",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        masks=[]
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det, metadata, category_id=category_id,
            )
            segm = coco_obj.segmentation #get the polygons
            height=metadata.height
            width=metadata.width
            #print(coco_obj)
            rles = maskUtils.frPyObjects(segm, height, width) #get RLE mask
            rle = maskUtils.merge(rles)
            mascara = maskUtils.decode(rle) #Transform RLEs masks in bitmaps    
            mascara = torch.tensor(mascara, dtype=torch.uint8) #make the bitmap a tensor
            masks.append(mascara) #Join mask bitmap tensors
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        #print(type(masks))
        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        #Masl
        masks=torch.stack(masks)
        #newMask = torch.cat(newMask, newMask.shape[0]) 
        target["masks"] = torch.as_tensor(masks, dtype=torch.uint8)
        #target["masks"]=masks

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [ ]:
IMAGES_DIR= root_path+"/dataset/inference_images"
JSON_DIR = root_path+"/dataset/labels.json"

NAME="C"
data = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    name = NAME,
    data_path=IMAGES_DIR,
    labels_path=JSON_DIR,
    include_id=True,
    label_field="",
    label_types="segmentations",
    extra_attrs=True,
)


In [ ]:
train_transforms = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5)])
test_transforms = T.Compose([T.ToTensor()])
car_dataset= FiftyOneTorchDataset(data)
print(car_dataset)
classes=car_dataset.get_classes()
print(classes)
print(len(classes))

torch_dataset_test = FiftyOneTorchDataset(data, test_transforms, 
        classes=classes)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device %s" % device)
img, _= torch_dataset_test[7]
print(type(img))
model.eval()
with torch.no_grad():
  prediction=model([img.to(device)])